## Scrape cars on sale on various trader websites

Approach is different for this one. All of the trader's car can be listed on one page. I've simply loaded the entire page and saved it, with picture files. This is what I'm parsing. 

This could be set up to be done on a regular basis to identify new cars, sold cars, and so on. 

Ran this script on:
* 09/12/2016
* 09/15/2016
* 09/16/2016
* 09/18/2016

Issues:
* Need to fix Alfa --> Alfa Romeo

In [116]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import sqlite3
import time

%load_ext sql
%config SqlMagic.autopandas=True
%matplotlib inline

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [117]:
%qtconsole

In [118]:
# Connect to dbs
conn = sqlite3.connect('data/sales.db')


In [119]:
source = 'www.vroom.com'
trader = 'http://www.vroom.com/catalog'
browser = webdriver.Chrome()
browser.get(trader)

In [120]:
#First get total number of listing today
# Will scroll down until we get that number of results
soup = BeautifulSoup(browser.page_source, "lxml")
num_results = int(soup.find_all("div", class_="results")[0].text.split()[0].replace(',',''))
print(num_results)

2386


In [122]:
# Scroll down the page until entire catalog (num_results) is loaded.
# Sometimes the site becomes unresponsive so set num_iter to stop it after a while.
num_iter = 0
num_cars = 0
while num_cars < num_results - 5 and num_iter < 10:
    for x in range(0,50):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.random()*.5)

    soup = BeautifulSoup(browser.page_source, "lxml")
    cars = soup.find_all("div", class_="car-new")
    # carsSP = soup.find_all("div", class_="car-new is-sale-pending")
    num_cars = len(cars)# + len(carsSP)
    print(num_cars)
    num_iter = num_iter + 1

876
1551
2151
2387


In [123]:
#Parse info
all_cars = pd.DataFrame()
date = '2016-09-18'

# soup = BeautifulSoup(open('./data/VR_scrape/VR09122016.html'),"lxml")
# cars = soup.find_all("div", class_="car-new")

for car in cars:
    try:
        name = car.find_all("div", class_="info")[0].find_all("div", class_="name")[0].text.split()
        if name[1] == 'Land' and name[2] == 'Rover':
            year,make,model = name[0],' '.join(name[1:3]),' '.join(name[3:])
#             print(make,model)
        else:
            year,make,model = name[0],name[1],' '.join(name[2:])
        trim = car.find_all("div", class_="trim")[0].text
        mileage = int(car.find_all("div", class_="mileage")[0].text.replace(',','').replace(' Miles',''))
        price = int(car.find_all("div", class_="price")[0].text.replace('$','').replace(',',''))
        img_file = car.find_all("img")[0]['src']
        all_cars = all_cars.append(pd.DataFrame(
            [[date, 'Used',year, make,model,trim,mileage,price,img_file,source]]),
                                      ignore_index = True)
    except:
        pass
    
        

In [124]:
# Add to sqlite table salesVR

all_cars.columns = ['date','new_used', 'year', 'make', 'model','trim', 'mileage', 'price', 'img_file', 'source']    
index_max = conn.execute('SELECT MAX(line_id) FROM salesVR').fetchall()
if index_max[0][0] is not None:
    all_cars.index = all_cars.index + 1 + index_max[0][0]
all_cars.to_sql('salesVR',conn,flavor='sqlite',if_exists='append',index_label='line_id')